# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [123]:
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *

In [124]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()

In [125]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [126]:
song_files = get_files('./data/')

In [127]:
song_files[1:5]

['/Users/ziyad/Downloads/project_template/data/log_data/2018/11/2018-11-23-events.json',
 '/Users/ziyad/Downloads/project_template/data/log_data/2018/11/2018-11-18-events.json',
 '/Users/ziyad/Downloads/project_template/data/log_data/2018/11/2018-11-04-events.json',
 '/Users/ziyad/Downloads/project_template/data/log_data/2018/11/2018-11-01-events.json']

In [128]:
import json 

In [129]:
filepath = []

In [130]:
for song in song_files:
    with open(song) as f:
        for line in f:
            filepath.append(json.loads(line))
    

In [134]:
filepath[0]

{'artist': 'Frumpies',
 'auth': 'Logged In',
 'firstName': 'Anabelle',
 'gender': 'F',
 'itemInSession': 0,
 'lastName': 'Simpson',
 'length': 134.47791,
 'level': 'free',
 'location': 'Philadelphia-Camden-Wilmington, PA-NJ-DE-MD',
 'method': 'PUT',
 'page': 'NextSong',
 'registration': 1541044398796.0,
 'sessionId': 455,
 'song': 'Fuck Kitty',
 'status': 200,
 'ts': 1541903636796,
 'userAgent': '"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.125 Safari/537.36"',
 'userId': '69'}

In [136]:
df = pd.DataFrame.from_dict(filepath)
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,...,num_songs,artist_id,artist_latitude,artist_longitude,artist_location,artist_name,song_id,title,duration,year
0,Frumpies,Logged In,Anabelle,F,0.0,Simpson,134.47791,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Kenny G with Peabo Bryson,Logged In,Anabelle,F,1.0,Simpson,264.75057,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Biffy Clyro,Logged In,Anabelle,F,2.0,Simpson,189.83138,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,None,Logged In,Lily,F,0.0,Burns,NaN,free,"New York-Newark-Jersey City, NY-NJ-PA",GET,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,HIM,Logged In,Lily,F,1.0,Burns,212.06159,free,"New York-Newark-Jersey City, NY-NJ-PA",PUT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [137]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8129 entries, 0 to 8128
Data columns (total 28 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   artist            6820 non-null   object 
 1   auth              8056 non-null   object 
 2   firstName         7770 non-null   object 
 3   gender            7770 non-null   object 
 4   itemInSession     8056 non-null   float64
 5   lastName          7770 non-null   object 
 6   length            6820 non-null   float64
 7   level             8056 non-null   object 
 8   location          7770 non-null   object 
 9   method            8056 non-null   object 
 10  page              8056 non-null   object 
 11  registration      7770 non-null   float64
 12  sessionId         8056 non-null   float64
 13  song              6820 non-null   object 
 14  status            8056 non-null   float64
 15  ts                8056 non-null   float64
 16  userAgent         7770 non-null   object 


## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [143]:
songs = df[['song_id','title','artist_id','year','duration']].dropna()

In [144]:
songs

,song_id,title,artist_id,year,duration
8056,SONHOTT12A8C13493C,Something Girls,AR7G5I41187FB4CE6C,1982.0,233.40363
8057,SOIAZJW12AB01853F1,Pink World,AR8ZCNI1187B9A069B,1984.0,269.81832
8058,SOFSOCN12A8C143F5D,Face the Ashes,ARXR32B1187FB57099,2007.0,209.60608
8059,SOHKNRJ12A6701D1F8,Drop of Rain,AR10USD1187B99F3F1,0.0,189.57016
8060,SOQHXMF12AB0182363,Young Boy Blues,ARGSJW91187B9B1D6B,0.0,218.77506
...,...,...,...,...,...
8124,SOLEYHO12AB0188A85,Got My Mojo Workin,ARAGB2O1187FB3A161,0.0,338.23302
8125,SOGVQGJ12AB017F169,Ten Tonne,AR62SOJ1187FB47BB5,2005.0,337.68444
8126,SOFFKZS12AB017F194,A Higher Place (Album Version),ARBEBBY1187B9B43DB,1994.0,236.17261
8127,SOFNOQK12AB01840FC,Kutt Free (DJ Volume Remix),ARNNKDK1187B98BBD5,0.0,407.37914


In [154]:
song_data = songs.loc[8056]

In [158]:
song_list=list(song_data)

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [160]:
songplay_table_drop = ("""DROP TABLE SONGS""")

In [161]:
cur.execute(songplay_table_drop)
conn.commit()

In [162]:
songplay_table_create = ("""CREATE TABLE  IF NOT EXISTs SONGS(song_id varchar(50),title varchar(50),artist_id varchar(50),year varchar(50),duration varchar(50)) 
""")
song_table_insert1 = 'INSERT INTO SONGS VALUES(%s,%s,%s,%s,%s)'

In [163]:
#conn.rollback()

#### create a table for songs

In [164]:
cur.execute(songplay_table_create)
conn.commit()

In [165]:
cur.execute(song_table_insert1, song_data)
conn.commit()

In [167]:
song_data

song_id      SONHOTT12A8C13493C
title           Something Girls
artist_id    AR7G5I41187FB4CE6C
year                       1982
duration                233.404
Name: 8056, dtype: object

In [171]:
run = 'SELECT * FROM songs'

In [172]:
cur.execute(run)
conn.commit()



In [173]:
row = cur.fetchone()
while row:
   print(row)
   row = cur.fetchone()


('SONHOTT12A8C13493C', 'Something Girls', 'AR7G5I41187FB4CE6C', '1982.0', '233.40363')


Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [177]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8129 entries, 0 to 8128
Data columns (total 28 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   artist            6820 non-null   object 
 1   auth              8056 non-null   object 
 2   firstName         7770 non-null   object 
 3   gender            7770 non-null   object 
 4   itemInSession     8056 non-null   float64
 5   lastName          7770 non-null   object 
 6   length            6820 non-null   float64
 7   level             8056 non-null   object 
 8   location          7770 non-null   object 
 9   method            8056 non-null   object 
 10  page              8056 non-null   object 
 11  registration      7770 non-null   float64
 12  sessionId         8056 non-null   float64
 13  song              6820 non-null   object 
 14  status            8056 non-null   float64
 15  ts                8056 non-null   float64
 16  userAgent         7770 non-null   object 


In [184]:
artist_columns = ['artist_id','artist_name','artist_location','artist_latitude','artist_longitude']
artist_data = df[artist_columns]
artist_data

,artist_id,artist_name,artist_location,artist_latitude,artist_longitude
0,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...
8124,ARAGB2O1187FB3A161,Pucho & His Latin Soul Brothers,,NaN,NaN
8125,AR62SOJ1187FB47BB5,Chase & Status,,NaN,NaN
8126,ARBEBBY1187B9B43DB,Tom Petty,"Gainesville, FL",NaN,NaN
8127,ARNNKDK1187B98BBD5,Jinx,Zagreb Croatia,45.80726,15.96760


In [188]:
artist_list=list(artist_data.loc[8056])

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [199]:
artist_table_create = "CREATE TABLE IF NOT EXISTS artists (artist_id varchar(50),artist_name varchar(50),artist_location varchar(50)\
,artist_latitude varchar(50),artist_longitude varchar(50))"

In [204]:
artist_table_insert1 = "INSERT INTO artists VALUES (%s,%s,%s,%s,%s)"

In [206]:
#conn.rollback()

In [202]:
cur.execute(artist_table_create)
conn.commit()

In [207]:
cur.execute(artist_table_insert1, artist_list)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [210]:
log_files = get_files('./data/log_data/')

In [212]:
log_files[0]

'/Users/ziyad/Downloads/project_template/data/log_data/2018/11/2018-11-11-events.json'

In [218]:
logs = []
for log in log_files:
    with open(log) as f:
        for line in f:
            logs.append(json.loads(line))


In [220]:
logs

TypeError: 'list' object is not callable

In [256]:
df = pd.DataFrame.from_dict(logs)
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Frumpies,Logged In,Anabelle,F,0,Simpson,134.47791,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,Fuck Kitty,200,1541903636796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
1,Kenny G with Peabo Bryson,Logged In,Anabelle,F,1,Simpson,264.75057,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,By The Time This Night Is Over,200,1541903770796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
2,Biffy Clyro,Logged In,Anabelle,F,2,Simpson,189.83138,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,God & Satan,200,1541904034796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
3,None,Logged In,Lily,F,0,Burns,NaN,free,"New York-Newark-Jersey City, NY-NJ-PA",GET,Home,1.540621e+12,456,None,200,1541910841796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",32
4,HIM,Logged In,Lily,F,1,Burns,212.06159,free,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540621e+12,456,Beautiful,200,1541910973796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",32


## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [257]:
df = df.loc[df['page']=='NextSong']
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Frumpies,Logged In,Anabelle,F,0,Simpson,134.47791,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,Fuck Kitty,200,1541903636796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
1,Kenny G with Peabo Bryson,Logged In,Anabelle,F,1,Simpson,264.75057,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,By The Time This Night Is Over,200,1541903770796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
2,Biffy Clyro,Logged In,Anabelle,F,2,Simpson,189.83138,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,455,God & Satan,200,1541904034796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69
4,HIM,Logged In,Lily,F,1,Burns,212.06159,free,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540621e+12,456,Beautiful,200,1541910973796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",32
5,Matmos,Logged In,Joseph,M,0,Gutierrez,1449.11628,free,"Columbia, SC",PUT,NextSong,1.540809e+12,284,Supreme Balloon,200,1541911006796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_3...",75


In [258]:
from datetime import datetime


In [261]:
t = df['ts'].apply(lambda x : datetime.fromtimestamp(x/1000.0))
t.head()

0   2018-11-11 05:33:56.796
1   2018-11-11 05:36:10.796
2   2018-11-11 05:40:34.796
4   2018-11-11 07:36:13.796
5   2018-11-11 07:36:46.796
Name: ts, dtype: datetime64[ns]

In [267]:
hour = t.dt.hour
day = t.dt.day
weekofyear = t.dt.weekofyear
month = t.dt.month
year = t.dt.year
dayofweek = t.dt.dayofweek

In [280]:
time_data = list(zip(hour,day,weekofyear,month,year,dayofweek))
column_labels = ('hour','day','weekofyear','month','year','dayofweek')

In [284]:
time_df = pd.DataFrame(data=time_data,columns=column_labels)
time_df.head()

,hour,day,weekofyear,month,year,dayofweek
0,5,11,45,11,2018,6
1,5,11,45,11,2018,6
2,5,11,45,11,2018,6
3,7,11,45,11,2018,6
4,7,11,45,11,2018,6


#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [ ]:
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [ ]:
user_df = 

#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [ ]:
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [ ]:
for index, row in df.iterrows():

    # get songid and artistid from song and artist tables
    results = cur.execute(song_select, (row.song, row.artist, row.length))
    songid, artistid = results if results else None, None

    # insert songplay record
    songplay_data = ()
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [ ]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.